# Resource estimation for simulating a 2D Ising Model Hamiltonian

In this Python+Q# notebook we demonstrate how to estimate the resources for quantum dynamics,
specifically the simulation of an Ising model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D
qubit architecture with nearest-neighbor connectivity.

First, we load the necessary Python packages.

In [ ]:
import qsharp
import pandas as pd

## Background: 2D Ising model

The Ising model is a mathematical model of ferromagnetism in a lattice (in our case a 2D square lattice) with two kinds of terms in the Hamiltonian: (i) an interaction term between adjacent sites and (ii) an external magnetic field acting at each site. For our purposes, we consider a simplified version of the model where the interaction terms have the same strength and the external field strength is the same at each site.
Formally, the Ising model Hamiltonian on an $N \times N$ lattice we consider is formulated as:

$$
H = \underbrace{-J \sum_{i, j} Z_i Z_j}_{B} + \underbrace{g \sum_j X_j}_{A}
$$
where $J$ is the interaction strength, $g$ is external field strength.

The time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= t/\Delta$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2). Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper functions

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B$ gives:
$$
U_4(\Delta) = e^{-iAp\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iAp\Delta/2}
$$

The above equation with $11$ exponential terms works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $10t+1$ exponential terms for $e^{-iHt}$.

The function below creates a sequence containing the constant factors that will be applied with $A$ and $B$, respectively, in the exponential sequence of the above formula.

In [ ]:
%%qsharp
function SetAngleSequence(p : Double, dt : Double, J : Double, g : Double) : Double[] {

    let len1 = 3;
    let len2 = 3;
    let valLength = 2*len1+len2+1;
    mutable values = [0.0, size=valLength];

    let val1 = J*p*dt;
    let val2 = -g*p*dt;
    let val3 = J*(1.0 - 3.0*p)*dt/2.0;
    let val4 = g*(1.0 - 4.0*p)*dt/2.0;

    for i in 0..len1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1;
        }
        else {
            set values w/= i <- val2;
        }

    }

    for i in len1+1..len1+len2 {
        if (i % 2 == 0) {
            set values w/= i <- val3;
        }
        else {
            set values w/= i <- val4;
        }
    }

    for i in len1+len2+1..valLength-1 {
        if (i % 2 == 0) {
            set values w/= i <- val1;
        }
        else {
            set values w/= i <- val2;
        }
    }
    return values;
}

### Quantum operations

There are two kinds of Pauli exponentials needed for simulating the time evolution of an Ising Model:
- The transverse field $e^{-iX\theta}$ applied to each qubit for an angle $\theta$;
- $e^{-i (Z \otimes Z)\theta}$ applied to neighboring pairs of qubits in the lattice.

The operation below applies $e^{-iX\theta}$ on all qubits in the 2D lattice.

In [ ]:
%%qsharp
operation ApplyAllX(n : Int, qArr : Qubit[][], theta : Double) : Unit {
    // This applies `Rx` with an angle of `2.0 * theta` to all qubits in `qs`
    // using partial application
    for row in 0..n-1 {
        ApplyToEach(Rx(2.0 * theta, _), qArr[row]);
    }
}

The next operation below applies $e^{-i(Z \otimes Z)\theta}$ on overlapping pairs of neighboring qubits. Observe that unlike the previous case, it is not possible to simultaneously apply all the rotations in one go. For example, while applying the rotation on qubits at $(0, 0)$ and $(0, 1)$, it is not possible to also apply the rotation on qubits $(0, 1)$ and $(0, 2)$. Instead, we try to apply as many rotations as possible. This is broken up as follows:
- in the vertical (resp. horizontal) direction of the 2D lattice as chosen by `dir`,
- consider pairs starting with an even (resp. odd) index as given by `grp`;
- apply the exponential to all such pairs in the lattice.

In [ ]:
%%qsharp
operation ApplyDoubleZ(n : Int, m : Int, qArr : Qubit[][], theta : Double, dir : Bool, grp : Bool) : Unit {
    let start = grp ? 1 | 0;    // Choose either odd or even indices based on group number
    let P_op = [PauliZ, PauliZ];
    let c_end = dir ? m-1 | m-2;
    let r_end = dir ? m-2 | m-1;

    for row in 0..r_end {
        for col in start..2..c_end {    // Iterate through even or odd columns based on `grp`

            let row2 = dir ? row+1 | row;
            let col2 = dir ? col | col+1;

            Exp(P_op, theta, [qArr[row][col], qArr[row2][col2]]);
        }
    }
}


The next operation puts everything together and calls the operations needed to
simulate the Ising model Hamiltonian using a fourth order product formula.
Observe that the `ApplyDoubleZ` operation is called four times for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `N1`, `N2`: row and column size for the lattice.
- `J`, `g`: parameters by which the Hamiltonian terms are scaled.
- `totTime`: the number of Trotter steps.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation IsingModel2DSim(N1 : Int, N2 : Int, J : Double, g : Double, totTime : Double, dt : Double) : Unit {
    
    use qs = Qubit[N1*N2];
    let qubitArray = Chunks(N2, qs); // qubits are re-arranged to be in an N1 x N2 array

    let p = 1.0 / (4.0 - 4.0^(1.0 / 3.0));
    let t = Ceiling(totTime / dt);

    let seqLen = 10 * t + 1;

    let angSeq = SetAngleSequence(p, dt, J, g);

    for i in 0..seqLen - 1 {
        let theta = (i==0 or i==seqLen-1) ? J*p*dt/2.0 | angSeq[i%10];

        // for even indexes
        if i % 2 == 0 {
            ApplyAllX(N1, qubitArray, theta);
        } else {
            // iterate through all possible combinations for `dir` and `grp`.
            for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
                ApplyDoubleZ(N1, N2, qubitArray, theta, dir, grp);
            }
        }
    }
}


## Getting logical resource counts

For the purpose of generating the rQOPS for some target runtime, it suffices to obtain the logical resource estimates to simulate the Heisenberg model Hamiltonian. We consider three problem instances with lattice sizes $\{10 \times 10, 20 \times 20, 30 \times 30\}$ with $J = g = 1.0$. These instances are simulated for a total time of $L$ seconds for lattice size $L$, with step size `dt`$ = 0.9$, and overall probability of failure $\varepsilon = 0.01$. Any one of the six pre-defined qubit parameters will do to obtain the logical coounts and in this notebook we choose a Majorana based qubit with the `floquet code`.

In [ ]:
N1 = [10, 20, 30]
N2 = [10, 20, 30]
totTime = [10.0, 20.0, 30.0]
J = 1.0
g = 1.0
dt = 0.9

We submit a resource estimation job with all the problem instances sequentially and collect the estimates in `results`.

In [ ]:
results = []
for i in range(3):
    qsharp_string = f"IsingModel2DSim({N1[i]}, {N2[i]}, {J}, {g}, {totTime[i]}, {dt})"

    result = qsharp.estimate(qsharp_string, params={"errorBudget": 0.01, "qubitParams": {"name": "qubit_maj_ns_e6"}, "qecScheme": {"name": "floquet_code"}, "constraints": {"logicalDepthFactor": 4}})
    results.append(result)

To see the complete information provided when invoking the resource estimator, we output the result for the $10 \times 10$ lattice by displaying `results[0]`

In [ ]:
# Displaying estimates for 10x10 lattice size.
results[0]
# Change index to 1 (resp. 2) for 20x20 (resp. 30x30) lattice size.

## Visualizing and understanding the results

### Result summary table

In [ ]:
# Define function to display information in summary format
def get_summary_table(results, labels):
    logical_qubits = []
    logical_depth = []
    t_states = []
    code_distance = []
    t_factories = []
    t_factory_fraction = []
    physical_qubits = []
    rqops = []
    runtime = []
    logical_error = []

    for i in range(3): 
        logical_qubits.append(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'])
        logical_depth.append(results[i]['physicalCountsFormatted']['logicalDepth'])
        t_states.append(results[i]['physicalCountsFormatted']['numTstates'])
        t_factories.append(results[i]['physicalCounts']['breakdown']['numTfactories'])
        logical_error.append(results[i]['physicalCountsFormatted']['requiredLogicalQubitErrorRate'])
        physical_qubits.append(results[i]['physicalCountsFormatted']['physicalQubits'])
        rqops.append(results[i]['physicalCountsFormatted']['rqops'])
        runtime.append(results[i]['physicalCountsFormatted']['runtime'])
        code_distance.append(results[i]['logicalQubit']['codeDistance'])
        t_factory_fraction.append(results[i]['physicalCountsFormatted']['physicalQubitsForTfactoriesPercentage'])

    data = pd.DataFrame()
    pd.options.display.float_format = '{:.2E}'.format
    data['Logical qubits'] = logical_qubits
    data['Logical depth'] = logical_depth
    data['Logical error'] = logical_error
    data['T states'] = t_states
    # data['T states'] = data['T states'].astype('float64')
    data['Code Distance'] = code_distance
    data['T factories'] = t_factories
    data['T factory fraction'] = t_factory_fraction
    data['Physical qubits'] = physical_qubits
    data['rQOPS'] = rqops
    data['Physical runtime'] = runtime
    data.index = labels

    return data

# Display summarized information for all problem instances
labels = ["Isi10", "Isi20", "Isi30"]
table = get_summary_table(results, labels)
table

> Note that in general, there is a trade-off between the logical depth and number of T factories used. 

> To ensure that T factories do not dominate the resource requirements, we set the `logical_depth_factor`${}=4$ adding some number of `noops` to increase the logical depth.

### Getting the target rQOPS

While the resource estimator generates a runtime for the given hardware profile, we want to set a target runtime of 2 days i.e., 172800 seconds to obtain a practical quantum advantage. We collect the previous results to compute the corresponding target rQOPS as 
$$ \text{Target rQOPS} = \frac{\text{Logical qubits}\cdot\text{Logical Depth}}{\text{Target runtime}}$$

In [ ]:
def get_target_rqops(results, labels):

    target_runtime = 172800
    logical_qubits = []
    logical_depth = []
    target_rqops = []
    logical_error = []

    for i in range(3): 
        logical_qubits.append(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'])
        logical_depth.append(results[i]['physicalCountsFormatted']['logicalDepth'])
        logical_error.append(results[i]['physicalCountsFormatted']['requiredLogicalQubitErrorRate'])
        target_rqops.append(round(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'] * results[i]['physicalCounts']['breakdown']['logicalDepth'] / target_runtime))

    data = pd.DataFrame()
    pd.options.display.float_format = '{:.2E}'.format
    data['Logical qubits'] = logical_qubits
    data['Logical depth'] = logical_depth
    data['Logical error'] = logical_error
    data['Target rQOPS'] = target_rqops
    data.index = labels

    return data

rQOPS_table = get_target_rqops(results, labels)
rQOPS_table


## Next steps

Feel free to use this notebook as a starting point for your own experiments.  For
example, you can

* explore how the results change considering other problem instances of the Heisenberg model
* explore space- and time-trade-offs by changing the value for
  `logical_depth_factor` or `max_t_factories`
* visualize these trade-offs with the space and time diagrams
* use other or customized qubit parameters